## Step 3 - LLM interaction using Hierarchical Approach

In this notebook, we will use the Hierarchical Approach to interact with the LLM model. The Hierarchical Approach is composed of two steps:
- Step 1: Generate an high-level sequence of actions to solve the task
- Step 2: From the high-level sequence of actions, generate a low-level/code-like sequence of actions

In [2]:
import pandas as pd
import os 
import numpy as np
import sklearn as sk
from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm 

Layer 1 Prompt 

In [3]:
nlp_plan_prompt =""" 
As an expert in robotic planning, your task is develop an plan for the robot to control the trajectory requested.
The user will provide the requested trajectory after keyword ##USER INPUT. Consider the ##ACTIONS during the creation of the plan.
The plan must respect the ##CONSTRAINTS when the robot is moving and the ##FORMAT of the output.

## ACTIONS  
- move straight (forward/backward)
- curve left/right (for example to create a circular path)
- rotate by any degree (change the orientation of the robot but not the coordinates)

## CONSTRAINTS 
- The field is a square measuring 11.0 meters in both the x and y directions.
- The robot's starting point is at x=0 and y=0, oriented along the positive x-axis (if the robot move forward it will go alongside the X). 
- Pay attention to not pass the limit of the fields (x < 0, x > 11, y < 0 ,y > 11)
- Each action must consider the current position and orientation of the robot.

## FORMAT
Return a JSON format with a single field : 
{
 "actions" : [list of actions]
 }

## EXAMPLE
user : create a semi-circle and came back to position x=0 and y=0

assistant :  {
    "actions" : ["move straight 1.0 meters to avoid the limit of the field to x=1 y=0", "curve left with radius 1.0 meters until a semi-circular path is created, reaching the point x = 1, y = 1.27 and oriented toward X negative", 
    "rotate 90 degrees anticlockwise to be oriented toward the Y negative", "move straight 1.27  meters to reach the point x = 1, y = 0 with orientation toward the Y negative",
    "rotate 90 degrees clockwise to be oriented toward the X negative", "move straight 1.0 meters to reach the point x = 0, y = 0"]
}

user : create a straight line 5 metres long and then go to the point x = 3 y = 5

assistant : {
    "actions" : ["move straight 5.0 meters to create a straight line, reaching the point x = 5 y = 0", "rotate 90 degrees anticlockwise to be oriented toward the Y positive", "move straight 5.0 meters to reach the point x = 5 y = 5",
    "rotate 90 degrees anticlockwise to be oriented toward the X negative", "move straight 2.0 meters to reach the point x = 3 y = 5"]
    }
"""

Layer 2 Prompt

In [4]:
detailed_step_composition_prompt = """
As an expert in robotics programming, your task is to convert the high-level plan into a sequence of actions that will be used in a simulation.
The user will provide the original request and the plan created for the robot. 
Pay attention to the mathematical calculations and the ## CONSTRAINTS of the robot's movements.

## CONSTRAINTS 
- The field is a square measuring 11.0 meters in both the x and y directions.
- The robot's starting point is at x=0 and y=0, oriented along the positive x-axis (if the robot move forward it will go alongside the X). 
- Pay attention to not pass the limit of the fields (x < 0, x > 11, y < 0 ,y > 11).
- To curve left or right, the robot must move forward and rotate simultaneously.


## FORMAT
Each action must be a JSON composed of the following fields:
- reasonings : the reasoning behind the conversion from high-level commands to actions.
- frontal_speed : the speed at which the robot moves forward in m/s 
- rotation : the rotational of the robot in degrees, positive for anticlockwise and negative for clockwise    
- time : the last of a single action in seconds

Return a list of JSON actions in the following format:
{
    "json_actions" : [list of actions]
}


## EXAMPLES

user : ## ORIGINAL REQUEST 
create a semi-circle and came back to position x=0 and y=0
## PLAN
{
    "actions" : ["move straight 1.0 meters to avoid the limit of the field to x=1 y=0", "curve left with radius 1.0 meters until a semi-circular path is created, reaching the point x = 1, y = 2.55 and oriented toward X negative", 
    "rotate 90 degrees anticlockwise to be oriented toward the Y negative", "move straight 1.27 meters to reach the point x = 1, y = 0 with orientation toward the Y negative",
    "rotate 90 degrees clockwise to be oriented toward the X negative", "move straight 1.0 meters to reach the point x = 0, y = 0"]
}

assistant : {
    "json_actions" : [
        {
            "reasoning" : "to move forward and avoid the limit of the field, front_speed is 1.0 m/s, rotation is 0.0 degrees and time is 1.0 seconds", 
            "frontal_speed" : 1.0, 
            "rotation" :0.0, 
            "time" : 1.0
        } ,
        {
            "reasoning" : "to curve left with radius 1.0 meters, front_speed is 1.0 m/s, anticlockwise rotation is 90.0 degrees and time is 1.57 seconds", 
            "frontal_speed" : 1.0, 
            "rotation" : 90.0, 
            "time" : 2.0
        } ,
        {
            "reasoning" : "to be oriented toward the Y negative, front_speed is 0.0 m/s, anticlockwise rotation is 90.0 degrees and time is 1.0 seconds", 
            "frontal_speed" : 0.0, 
            "rotation" : 90.0, 
            "time" : 1.0
        } ,
        {
            "reasoning" : "to reach the point x = 1, y = 0 from the previous position x = 1, y = 1.27, front_speed is 1.0 m/s, rotation is 0.0 degrees and time is 1.27 seconds", 
            "frontal_speed" : 1.0, 
            "rotation" : 0.0, 
            "time" : 1.27
        } ,
        {
            "reasoning" : "to be oriented toward the X negative, front_speed is 0.0 m/s, clockwise rotation is 90.0 degrees and time is 1.0 seconds", 
            "frontal_speed" : 0.0, 
            "rotation" : -90.0, 
            "time" : 1.0
        } ,   
        {
            "reasoning" : "to reach the point x = 0, y = 0 from the previous position x = 1, y = 0, front_speed is 1.0 m/s, rotation is 0.0 degrees and time is 1.0 seconds", 
            "frontal_speed" : 1.0, 
            "rotation" : 0.0, 
            "time" : 1.0
        } ,     
    ]
}
"""

In [5]:
from openai import OpenAI, AzureOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
import ast
import os 
from dotenv import load_dotenv
load_dotenv('.env.local')



def gpt_chat_completion(model : str ,messages : list[dict], temperature: float = 0.0, frequency_penalty: float = 0.0 , max_tokens: int = None): 
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY") )
    completion = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL") if model is None else model,
        messages=messages,
        temperature=temperature,
        seed = 42,
        frequency_penalty = frequency_penalty,
        #max_tokens=max_tokens,
        )
    return completion.choices[0].message.content


def gpt_chat_completion_azure(messages : list[dict], temperature: float = 0.0, frequency_penalty: float = 0.0 , max_tokens: int = None): 
    client = AzureOpenAI(api_key = os.getenv("AZURE_OPENAI_KEY"),  
    api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),)
    completion = client.chat.completions.create(
        

        model="gpt-4-turbo",
        messages=messages,
        temperature=temperature,
        seed = 42,
        frequency_penalty = frequency_penalty,
        #max_tokens=max_tokens,
        )
    return completion.choices[0].message.content

def get_embedding(text, model="large"):
    if model == 'large' : 
      emb_model = "text-embedding-3-large"
    elif model == 'small' :
        emb_model = "text-embedding-3-small"
    else: 
        emb_model = "text-embedding-ada-002"
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY") )
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=emb_model).data[0].embedding


def claude_completion(messages : list[tuple], model='claude-3-opus-20240229') : 
    llm = ChatAnthropic(model=model,
                    temperature=0,
                    max_tokens=1024,
                    api_key=os.getenv("ANTHROPIC_API_KEY"),
                    
                    )
    prompt = ChatPromptTemplate.from_messages(
        messages=messages
    )

    chain = prompt | llm
    msg = chain.invoke(messages)
    
    return msg.content

In [6]:
questions = [
    "create a rectangle of dimension X = 3 m and Y = 1 m",
	"crea un quadrato di lunghezza 2 metri",
	"crea un triangolo equilatero",
	"create a circle respecting the limits of the fields"  ,
	"reach x = 2.5 and y= 9.0",
	"go to point x=6 y=7 passing through x=2 and y=2",
	"create a square and than a triangle",
	"create a rectangle of x = 5 and y = 3 and than go to position x=9 and y=9",
	"create a U trajectory",
	"create a Z trajectory"  
]*3


In [ ]:
import logging
from tqdm import tqdm
from time import time
tot_time = 0
for model in ["gpt-3.5-turbo-0125", "gpt-4o"]:
    results = list()
    for  question in tqdm(questions) : 
        init_time = time()
        messages=[
        {"role": "system", "content": nlp_plan_prompt},
        {"role": "user", "content": "## USER INPUT\n" + question}
            ]
        logging.info("High level plan completed")
        high_level_plan = gpt_chat_completion(model, messages, temperature=0.0)
        json_plan = high_level_plan[ high_level_plan.find("{"): high_level_plan.rfind("}")+1]  
        second_layer = detailed_step_composition_prompt
        
        messages_second_layer = [
            {"role": "system", "content": second_layer},
            {"role": "user", "content": "## ORIGINAL REQUEST\n" + question + "\n## PLAN\n" + json_plan} 
        ]
        
        detailed_steps = gpt_chat_completion(model, messages_second_layer, temperature=0.0)
        end_time = time()
        tot_time += end_time - init_time
        
        results.append({"question" : question , "high_level_plan" : json_plan , "steps" :detailed_steps })
        print(f"Low level plan for question :  {question} !! COMPLETED")
    tot_time = tot_time / len(questions)
    print(f"Average time for model {model} : {tot_time}")
    df = pd.DataFrame(results)
    df.to_excel(f"{model}_results.xlsx", index=False)
